In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [19]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [29]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "acousticness",
    "danceability",
    "duration_ms",
    "energy",
    "explicit",
    "instrumentalness",
    "key",
    "liveness",
    "loudness",
    "mode",
    "popularity",
    "speechiness",
    "tempo",
    "valence",
    "year"
]

target = ["popularity"]

In [36]:
# Load the data
file_path = Path('clean_spotify.csv')
df = pd.read_csv(file_path)
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
###issued_mask = df['loan_status'] != 'Issued'
###df = df.loc[issued_mask]

# convert interest rate to numerical
###df['int_rate'] = df['int_rate'].str.replace('%', '')
###df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
###x = {'Current': 'low_risk'}   
###df = df.replace(x)

###x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
###df = df.replace(x)

###df.reset_index(inplace=True, drop=True)

df.head()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
0,0.04950,0.612,240400,0.807,0,0.017700,10,0.1010,-2.810,1,89,0.0336,124.053,0.398,2013
1,0.69500,0.445,244360,0.537,1,0.000017,4,0.0944,-8.532,0,88,0.0400,122.769,0.131,2013
2,0.00286,0.908,290320,0.669,1,0.000000,7,0.2370,-2.827,1,87,0.0738,112.238,0.662,2002
3,0.03020,0.949,284200,0.661,1,0.000000,5,0.0454,-4.244,0,86,0.0572,104.504,0.760,2000
4,0.03710,0.764,196520,0.705,0,0.000019,3,0.0943,-5.279,0,86,0.0278,101.003,0.672,2015


# Split the Data into Training and Testing

In [31]:
# Create our features
#X = YOUR CODE HERE
X = df.drop(columns='popularity')
X = pd.get_dummies(X)

# Create our target
#y = # YOUR CODE HERE
y = df.loc[:, target].copy()

In [32]:
X.describe()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,year
count,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000
mean,0.128173,0.667814,228594.973725,0.721549,0.276662,0.015372,5.369397,0.181726,-5.514082,0.553323,0.103783,120.158442,0.552966,2009.520350
std,0.172584,0.140608,39249.796103,0.152872,0.447463,0.088371,3.615270,0.140910,1.938950,0.497277,0.096148,26.990475,0.220845,5.875532
min,0.000019,0.129000,113000.000000,0.054900,0.000000,0.000000,0.000000,0.021500,-20.514000,0.000000,0.023200,60.019000,0.038100,1998.000000
25%,0.013500,0.581000,203506.000000,0.624000,0.000000,0.000000,2.000000,0.088400,-6.490000,0.000000,0.039700,98.986000,0.390000,2004.000000
50%,0.055800,0.676000,223186.000000,0.739000,0.000000,0.000000,6.000000,0.124000,-5.285000,1.000000,0.061000,120.028000,0.560000,2010.000000
75%,0.176000,0.765000,247946.000000,0.840000,1.000000,0.000069,8.000000,0.242000,-4.168000,1.000000,0.129000,134.199000,0.731000,2015.000000
max,0.976000,0.975000,484146.000000,0.999000,1.000000,0.985000,11.000000,0.853000,-0.276000,1.000000,0.576000,210.851000,0.973000,2020.000000


In [33]:
# Check the balance of our target values
y['popularity'].value_counts()

0     126
69     74
68     73
73     67
74     65
     ... 
16      1
17      1
18      1
24      1
89      1
Name: popularity, Length: 76, dtype: int64

In [34]:
# Split the data set into Train & Test values
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [35]:
#view ytrain counts
y_train.value_counts()

popularity
0             92
73            57
68            57
69            56
61            51
              ..
16             1
17             1
24             1
26             1
88             1
Length: 73, dtype: int64

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [25]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier

brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [26]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7885466545953005

In [30]:
# Create a DataFrame from the confusion matrix.
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[   71,    30],
       [ 2153, 14951]], dtype=int64)

In [31]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,71,30
Actual low_risk,2153,14951


In [18]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.70      0.87      0.06      0.78      0.60       101
   low_risk       1.00      0.87      0.70      0.93      0.78      0.62     17104

avg / total       0.99      0.87      0.70      0.93      0.78      0.62     17205



In [19]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
# Get importances and features
importances = brf.feature_importances_
cols = X.columns

# Store in a DataFrame
feature_importances_df = pd.DataFrame({'feature':cols, 'importance': importances})
feature_importances_df.head()

,feature,importance
0,loan_amnt,0.010580
1,int_rate,0.029670
2,installment,0.019802
3,annual_inc,0.014833
4,dti,0.017471


In [20]:
# Sort the DataFrame - Not in starter code
feature_importances_df.sort_values('importance', ascending=False)

,feature,importance
15,total_rec_prncp,0.078768
13,total_pymnt,0.058838
14,total_pymnt_inv,0.056256
16,total_rec_int,0.053555
20,last_pymnt_amnt,0.050033
...,...,...
19,collection_recovery_fee,0.000000
22,policy_code,0.000000
23,acc_now_delinq,0.000000
46,delinq_amnt,0.000000


### Easy Ensemble AdaBoost Classifier

In [33]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)

eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [34]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9316600714093861

In [23]:
# Create a DataFrame from the confusion matrix.
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[   93,     8],
       [  983, 16121]], dtype=int64)

In [35]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,93,8
Actual low_risk,983,16121


In [24]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

